#### Data Sampling

The objective of this exercise is convert to our JSON data into a tabular format. Causal modeling software usually requires a dataframe-like object (Ex: [DoWhy](https://github.com/microsoft/dowhy)). Therefore, we need to "flatten" the data in a meaningful way. 

In [1]:
import pathlib, json
import pprint as pp
import random
import pandas as pd
import matplotlib.pyplot as plt
JSON_PATH = pathlib.Path("./data/validated/")
validated = list(JSON_PATH.glob('*.json'))

To fit the data in memory, we subsample based on a few criteria. Here we have chosen year and action (e.g. "hacking")

In [51]:
#List of path objects
filtered = []
#In memory store of sampled data
content = []
#Subsampling the data

FILTER_YEAR = 2014

for entry in validated:
    with entry.open('r') as f:
        data = json.load(f)
        #Filter by action type 
        if not ('malware' in data['action'] or 'hacking' in data['action']):
            continue
        
        if data['security_incident'] != 'Confirmed':
            continue
            
        if data['timeline']['incident']['year'] < FILTER_YEAR:
            continue
        
        filtered.append(entry)
        content.append(data)
        
print("{0} entries containing hacking or malware after {1}.".format(len(filtered), FILTER_YEAR))

1144 entries containing hacking or malware after 2014.


In [52]:
# Random entry, just to give us an idea of what our data looks like.
pp.pprint(random.choice(content))

{'action': {'malware': {'result': ['Exfiltrate'],
                        'variety': ['Ransomware'],
                        'vector': ['Unknown']}},
 'actor': {'external': {'country': ['Unknown'],
                        'motive': ['Financial'],
                        'region': ['000000'],
                        'variety': ['Unaffiliated']}},
 'asset': {'assets': [{'amount': 1, 'variety': 'S - Database'}],
           'cloud': ['Unknown'],
           'total_amount': 1},
 'attribute': {'availability': {'duration': {'unit': 'Never'},
                                'variety': ['Obscuration']},
               'confidentiality': {'data': [{'amount': 500,
                                             'variety': 'Medical'}],
                                   'data_disclosure': 'Potentially',
                                   'data_total': 500,
                                   'data_victim': ['Patient'],
                                   'state': ['Stored']},
               'integrity':

In [65]:
#Flattened data so far. Each dot indicates another nested level.
df = pd.io.json.json_normalize(content)


##### TODO: 
* Merge 'malware' and 'hacking' attributes

Some are categorized as both malware and hacking. Maybe look into 'variety' attribute.
* Resolve lists into categorical data


Our list of keynames should look similar to [this](https://github.com/vz-risk/VCDB/blob/master/vcdb-keynames-real.txt)
                                                                                    

In [69]:

# df = df.loc[:,~df.columns.str.startswith('plus.pci')]
# df = df.loc[:, ~df.columns.str.endswith('notes', 'pci.')]
df

,confidence,incident_id,reference,schema_version,security_incident,source_id,summary,targeted,action.hacking.variety,action.hacking.vector,...,plus.pci.req_2,plus.pci.req_3,plus.pci.req_4,plus.pci.req_5,plus.pci.req_6,plus.pci.req_7,plus.pci.req_8,plus.pci.req_9,plus.security_maturity,actor.internal.notes
0,Medium,9eda6e80-de98-11e7-9424-7b935f12be98,http://wach.com/news/local/lexington-medical-c...,1.3.4,Confirmed,vcdb,Officials say an employee database was hacked ...,Unknown,[Unknown],[Web application],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,39B1FCE8-E94B-49DD-98AF-8B8E75F52F96,https://www.databreaches.net/quebec-liberals-s...,1.3.4,Confirmed,vcdb,default password exploited,NaN,[Brute force],[Web application],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,C40D9A65-15CA-4F96-8AD7-16580C90A4E1,https://www.databreaches.net/some-previously-u...,1.3.4,Confirmed,vcdb,phishing email,NaN,[Unknown],[Unknown],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Low,44293100-d2f9-11e8-9b3a-b50b161886e9,https://www.wfla.com/news/local-news/pdq-victi...,1.3.4,Confirmed,vcdb,PDQ said a hacker exploited part of their comp...,Unknown,[Unknown],[Unknown],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,High,4d331ca0-0b73-11e8-827c-bba53cbad290,http://katu.com/news/nation-world/report-uber-...,1.3.4,Confirmed,vcdb,Hackers stole the personal data of 57 million ...,Unknown,[Unknown],[Web application],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,Medium,6f972a80-b68a-11e8-b56a-011002460f8b,https://oag.ca.gov/system/files/CA-%20Notice%2...,1.3.4,Confirmed,vcdb,Phishing leads to email credential loss and reuse,Targeted,[Use of stolen creds],[Web application],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,NaN,5d3444c0-999a-11e8-bb21-7f662dc72866,https://indianexpress.com/article/cities/mumba...,1.3.4,Confirmed,vcdb,cyber fraudsters encrypted data belonging to a...,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,NaN,952FD03A-54B8-422E-8FD4-000EB0481C3E,http://www.databreaches.net/utorrent-forums-ha...,1.3.4,Confirmed,vcdb,"The BitTorrent Client uTorrent, established by...",NaN,[Unknown],[Web application],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,Low,396470d0-d959-11e7-a452-77d2890a67ae,https://www.databreaches.net/nhs-trust-cancels...,1.3.4,Confirmed,vcdb,Malware outbreak leads to suspension of operat...,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df_action = df.loc[:, df.columns.str.startswith('action')]

In [68]:
df_action

,action.hacking.variety,action.hacking.vector,action.hacking.notes,action.malware.variety,action.malware.vector,action.social.target,action.social.variety,action.social.vector,action.hacking.result,action.malware.result,...,action.error.notes,action.error.variety,action.error.vector,action.misuse.result,action.misuse.variety,action.misuse.vector,action.physical.result,action.hacking.cve,action.misuse.notes,action.physical.notes
0,[Unknown],[Web application],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[Brute force],[Web application],it was as easy as using a commonly used passwo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[Unknown],[Unknown],NaN,[Unknown],[Unknown],[End-user or employee],[Phishing],[Email],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[Unknown],[Unknown],NaN,"[Capture app data, Ram scraper]",[Unknown],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[Unknown],[Web application],NaN,NaN,NaN,NaN,NaN,NaN,"[Infiltrate, Exfiltrate]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,[Use of stolen creds],[Web application],NaN,NaN,NaN,[Unknown],[Phishing],[Email],[Infiltrate],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1140,NaN,NaN,NaN,[Ransomware],[Unknown],NaN,NaN,NaN,NaN,[Infiltrate],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,[Unknown],[Web application],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,NaN,NaN,NaN,[Unknown],[Unknown],NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
